In [2]:
from deepface import DeepFace

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    state, frame = cap.read()

    if not state:
        break

    # Flip the frame (mirror view)
    frame = cv2.flip(frame, 1)
    
    # Detecting Frame
    res = DeepFace.find(frame , db_path='D:\Smart_Attendance\database', enforce_detection = False , model_name= 'VGG-Face')
    
    if len(res[0]['identity'])>0:
        name = name = res[0]['identity'].iloc[0].replace('/', '\\').split('\\')[-2]
        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        w =  res[0]['source_w'][0]
        h = res[0]['source_h'][0]

        xmax = int(xmin + w)
        ymax = int(ymin + h)
        
        # capture face through Frame
        cv2.rectangle(frame , (xmin,ymin), (xmax,ymax),(255,0,0))
        
        #Text on Frame
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        cv2.putText(frame, name, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the webcam feed
    cv2.imshow('Attendance', frame)

    # Wait for 'q' key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### CSV File Integration

In [ ]:
import cv2
import os
import pandas as pd
from datetime import datetime
from deepface import DeepFace

# ==============================
# CONFIG
# ==============================
db_path = r"D:\Smart_Attendance\database"

# Today's CSV file name (changes daily)
today_date = datetime.now().strftime("%Y-%m-%d")
attendance_file = f"attendance_{today_date}.csv"

# Roll Number mapping (extend this dictionary as per your students)
roll_numbers = {
    "Ram": "101",
    "vaishnavi mule": "102",
    "Sujan" : "100"
}

# Create CSV for today if not exists
if not os.path.exists(attendance_file):
    df = pd.DataFrame(columns=["Name", "Roll No", "Entry Time"])
    df.to_csv(attendance_file, index=False)

# ==============================
# CAMERA & ATTENDANCE
# ==============================
cap = cv2.VideoCapture(0)
marked_students = set()   # To avoid duplicate entries per day

while True:
    state, frame = cap.read()
    if not state:
        break

    frame = cv2.flip(frame, 3)  # mirror view

    try:
        res = DeepFace.find(frame, db_path=db_path, enforce_detection=False, model_name='VGG-Face')

        if len(res[0]['identity']) > 0:
            # Extract student name from folder path
            name = res[0]['identity'].iloc[0].replace('/', '\\').split('\\')[-2]

            # Attendance logging
            if name not in marked_students:
                roll_no = roll_numbers.get(name, "Unknown")
                entry_time = datetime.now().strftime("%H:%M:%S")

                # Append to today's CSV
                df = pd.read_csv(attendance_file)
                df = pd.concat([df, pd.DataFrame([[name, roll_no, entry_time]], 
                                                 columns=["Name", "Roll No", "Entry Time"])], 
                                                 ignore_index=True)
                df.to_csv(attendance_file, index=False)

                marked_students.add(name)
                print(f"Attendance marked: {name} ({roll_no}) at {entry_time}")

            # Bounding box around face
            xmin = int(res[0]['source_x'][0])
            ymin = int(res[0]['source_y'][0])
            w = res[0]['source_w'][0]
            h = res[0]['source_h'][0]

            xmax = int(xmin + w)
            ymax = int(ymin + h)

            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
            cv2.putText(frame, name, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    except Exception as e:
        # If no face found, just skip
        pass

    cv2.imshow("Attendance", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### old version

In [ ]:
import cv2
import mysql.connector
from datetime import datetime
from deepface import DeepFace

# ==============================
# CONFIG
# ==============================
db_path = r"D:\Smart_Attendance\database"

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="MySQL@2025",
    database="smart_attendance"
)
cursor = conn.cursor()

# Track already marked students per lecture
marked_students = set()

# ==============================
# Function: Get current lecture
# ==============================
def get_current_lecture():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    today = now.date()

    query = """
        SELECT lecture_id, subject, course, year, classroom
        FROM lectures
        WHERE date = %s
        AND start_time <= %s
        AND end_time >= %s
        LIMIT 1
    """
    cursor.execute(query, (today, current_time, current_time))
    lecture = cursor.fetchone()
    return lecture  # (lecture_id, subject, course, year, classroom)

# ==============================
# CAMERA & ATTENDANCE
# ==============================
cap = cv2.VideoCapture(0)

while True:
    state, frame = cap.read()  # Captures frame from webcam.
    if not state:
        break

    frame = cv2.flip(frame, 1)  # mirror view
    lecture = get_current_lecture()

    if not lecture:
        cv2.putText(frame, "No Active Lecture", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Attendance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    lecture_id, subject, course, year, classroom = lecture

    try:
        res = DeepFace.find(frame, db_path=db_path, enforce_detection=False, model_name='VGG-Face')

        if len(res[0]['identity']) > 0:
            # Extract student name from folder path
            name = res[0]['identity'].iloc[0].replace('/', '\\').split('\\')[-2]

            # Get student_id from DB
            cursor.execute("SELECT student_roll_no FROM students WHERE name = %s", (name,))
            student_row = cursor.fetchone()

            if student_row:
                student_roll_no = student_row[0]

                # Avoid duplicate marking
                if (student_roll_no, lecture_id) not in marked_students:
                    entry_time = datetime.now().strftime("%H:%M:%S")

                    # Insert attendance
                    
                    # Insert into lecture_scans (raw log)
                    cursor.execute("""
                        INSERT INTO lecture_scans (lecture_id, student_roll_no, scan_no, appeared)
                        VALUES (%s, %s, %s, %s)
                    """ (lecture_id, student_roll_no, 1, 1))

                    

                    conn.commit()
                    marked_students.add((student_roll_no, lecture_id))

                    print(f"✅ Attendance marked: {name} ({student_roll_no}) in Lecture {lecture_id} at {entry_time}")

            # Draw bounding box
            xmin = int(res[0]['source_x'][0])
            ymin = int(res[0]['source_y'][0])
            w = res[0]['source_w'][0]
            h = res[0]['source_h'][0]
            xmax, ymax = xmin + w, ymin + h

            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
            cv2.putText(frame, name, (xmin, ymin - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    except Exception as e:
        pass  # skip if no face

    cv2.imshow("Attendance", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cursor.close()
conn.close()
